In [ ]:
ls

glove.6B.50d.txt  sample_data/  test.csv  train.csv


In [ ]:
pip install gensim

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import gensim.downloader as api
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import re, nltk
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import csv
import nltk


In [ ]:
from bs4 import BeautifulSoup

def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text()
    return cleaned_text

html_text = """["why doesn't an optical mouse work on a glass table? or even on some surfaces? optical mice use an led and a camera to rapidly capture images of the surface beneath the mouse.  the infomation from the camera is analyzed by a dsp (digital signal processor) and used to detect imperfections in the underlying surface and determine motion. some materials, such as glass, mirrors or other very shiny, uniform surfaces interfere with the ability of the dsp to accurately analyze the surface beneath the mouse.  \\nsince glass is transparent and very uniform, the mouse is unable to pick up enough imperfections in the underlying surface to determine motion.  mirrored surfaces are also a problem, since they constantly reflect back the same image, causing the dsp not to recognize motion properly. when the system is unable to see surface changes associated with movement, the mouse will not work properly. ", 'what is the best off-road motorcycle trail ? long-distance trail throughout ca i hear that the mojave road is amazing!<br />\\nsearch for it online. ', 'what is trans fat? how to reduce that? i heard that tras fat is bad for the body.  why is that? where can we find it in our daily food? trans fats occur in manufactured foods during the process of partial hydrogenation, when hydrogen gas is bubbled through vegetable oil to increase shelf life and stabilize the original polyunsatured oil. the resulting fat is similar to saturated fat, which raises "bad" ldl cholesterol and can lead to clogged arteries and heart disease. \\nuntil very recently, food labels were not required to list trans fats, and this health risk remained hidden to consumers. in early july, fda regulations changed, and food labels will soon begin identifying trans fat content in processed foods. ', 'how many planes fedex has? i heard that it is the largest airline in the world according to the www.fedex.com web site:\\nair fleet<br />\\n  <br />\\n670 aircraft, including: <br />\\n47 airbus a300-600s  17 boeing dc10-30s  <br />\\n62 airbus a310-200/300s  36 boeing md10-10s  <br />\\n2 atr 72s  5 boeing md10-30s  <br />\\n29 atr 42s  57 boeing md11s  <br />\\n18 boeing 727-100s  10 cessna 208as  <br />\\n94 boeing 727-200s  246 cessna 208bs  <br />\\n30 boeing dc10-10s  17 fokker f-27s ', 'in the san francisco bay area, does it make sense to rent or buy ? the prices of rent and the price of buying does not make sense to me, mostly the rent will not cover the mortgage . is it better to rent a house or to buy? renting vs buying depends on your goals. <br />\\ngenerally thinking is that buying is better b/c the payments that would go into the rent start building equity in your home. the govt also incentivizes you to buy by making your property tax payments and mortgage interest payments tax deductible.\\nhaving said that current housing status in the bay area is such that housing cost to purchase is relatively high and rental prices (compared to ownership cost) are relatively low (relative to the rest of the country). it makes lese sense to buy vs. other places.\\nbottom line you should base your decision on whether you think the market will keep going up or not. the other numbers tend to even out, the main gain or loss in buying comes from appreciation/depreciation. ']"""

cleaned_text = remove_html_tags(html_text)
print(cleaned_text)


In [ ]:
texts, labels = [], []
data_path = "./train.csv"
with open(data_path) as csv_file:
    reader = csv.reader(csv_file, quotechar='"')
    for idx, line in enumerate(reader):
        text = ""
        for tx in line[1:]:
            feature_cleaned = tx.replace('"', '').replace('\\n', ' ').replace('\\', ' ').replace(',', ' ')
            text += feature_cleaned.lower()
            text += " "
        label = int(line[0]) - 1
        texts.append(text)
        labels.append(label)

In [ ]:
print(texts[1])

what is the best off-road motorcycle trail ? long-distance trail throughout ca i hear that the mojave road is amazing!<br /> search for it online. 


In [ ]:
nltk.download('punkt')

def split_into_sentences(text):
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text()
    sentences = nltk.sent_tokenize(cleaned_text)
    cleaned_sentences = []
    for sentence in sentences:
        cleaned_sentence = re.sub(r'[?.!]+$', '', sentence)
        cleaned_sentences.append(cleaned_sentence.strip())
    return cleaned_sentences

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower()
    nltk.download('punkt')
    words = nltk.word_tokenize(sentence)
    words = [word for word in words if re.match(r'^[a-zA-Z]+$', word)]
    return words

In [ ]:
def WordEmbedding(sentence):
    words = preprocess_sentence(sentence)
    sentence_embedding = []
    for word in words:
        if word in vocab:
          word_embedding = torch.tensor(vocab[word]).float()  # Load embedding from Glove model
        else:
          word_embedding = torch.randn(50)
        sentence_embedding.append(word_embedding)

    if sentence_embedding:
        return torch.stack(sentence_embedding)  # Convert list of tensors to a single tensor
    else:
        return torch.randn(1, 50)


In [ ]:
texts_cleaned = [split_into_sentences(text) for text in texts]

In [ ]:
print(texts_cleaned[1])

['what is the best off-road motorcycle trail', 'long-distance trail throughout ca i hear that the mojave road is amazing', 'search for it online']


In [ ]:
doc = texts_cleaned[5]

In [ ]:
sentences = [sent.strip() for sent in doc]
sentences = [WordEmbedding(sentence) for sentence in sentences]
doc_length = len(sentences)
sent_length = max([len(sent) for sent in sentences])


In [ ]:
data = []
for line in lines:
    part = line.strip().split("\t")
    sample = part[0].strip('][').split(',')
    label = sample[0].strip('"')
    feature = ','.join(sample[1:]).strip('"')
    feature_cleaned = feature.replace('"', '').replace('\\n', ' ').replace('\\', '').replace(',', ' ').replace('?', ' ')
    data.append([label, feature_cleaned])

df = pd.DataFrame(data, columns=["Label", "Feature"])



In [ ]:
print(df['Feature'][0])

why doesn't an optical mouse work on a glass table  or even on some surfaces  Optical mice use an LED and a camera to rapidly capture images of the surface beneath the mouse.  The infomation from the camera is analyzed by a DSP (Digital Signal Processor) and used to detect imperfections in the underlying surface and determine motion. Some materials  such as glass  mirrors or other very shiny  uniform surfaces interfere with the ability of the DSP to accurately analyze the surface beneath the mouse.   Since glass is transparent and very uniform  the mouse is unable to pick up enough imperfections in the underlying surface to determine motion.  Mirrored surfaces are also a problem  since they constantly reflect back the same image  causing the DSP not to recognize motion properly. When the system is unable to see surface changes associated with movement  the mouse will not work properly.


In [ ]:
file_path = './glove.6B.50d.txt'

vocab = {}

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split()
        word = parts[0]

        vector = torch.tensor([float(val) for val in parts[1:]])
        vocab[word] = vector

In [ ]:
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(han.parameters(), lr=0.001)

# number of epochs
num_epochs = 2

for epoch in range(num_epochs):
    running_loss = 0.0
    for doc, label in zip(texts_cleaned, labels):
        sentences = [sent.strip() for sent in doc]
        sentences = [WordEmbedding(sentence) for sentence in sentences]
        doc_length = len(sentences)
        sent_length = max([len(sent) for sent in sentences])
        han = HAN(sent_length, doc_length, 50, 20, 40, 10)

        optimizer.zero_grad()

        outputs = han(doc)

        label_index = int(label)
        num_classes = 10  # Số lượng lớp (nhãn)
        label_tensor = torch.zeros(num_classes)
        label_tensor[label_index-1] = 1
        loss = criterion(outputs, label_tensor)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss}")


In [ ]:
print(vocab['i'])

In [ ]:
sentences = ["The final week of the NBA regular season has plenty of play-in tournament drama on. ",
             "None of the four play-in spots in the Eastern and Western Conferences are set going into games",
             "The Eastern Conference race is far more since the Indiana, opponents left on their respective schedule"]

In [ ]:
sentences = [(WordEmbedding(sentence)) for sentence in sentences]
sentences = pad_sequence([torch.tensor(sentence) for sentence in sentences], batch_first=True)   # padding


In [ ]:
print((sentences).shape)

In [ ]:
class WordEncoder(nn.Module):
    def __init__(self, embedding_dim, hidden_units):
        super(WordEncoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_units = hidden_units
        self.gru = nn.GRU(embedding_dim, hidden_units, bidirectional=True)
        self.hidden_state = torch.zeros(2, hidden_units)
    def forward(self, sentence_embedding):   # hidden state = [2, batch_size, hidden_size] cuz it's bidirectional
        outputs, _ = self.gru(sentence_embedding, self.hidden_state)
        return outputs   # [sentence_length, 2 * hidden_units]

In [ ]:
class WordAttention(nn.Module):
    def __init__(self, input_size, hidden_size):
      super(WordAttention, self).__init__()
      self.hidden_size = hidden_size
      self.score_weight_matrix = nn.Linear(input_size, hidden_size, bias=True)
      self.softmax = nn.Softmax(dim=1)
      self.context_vector = nn.Parameter(torch.randn(1, hidden_size))

    def forward(self, output_tensor):
      u_i = torch.tanh(self.score_weight_matrix(output_tensor))   # [batch_size, T, hidden_size]
      dot_product = torch.matmul(u_i, self.context_vector.transpose(0, 1))  # [batch_size, T, hidden_size]
      a_i = nn.Softmax(dim=0)(dot_product)  # [batch_size, T, hidden_size]
      s_i = (a_i * output_tensor).sum(dim=1)
      return s_i

In [ ]:
class SentenceEncoder(nn.Module):
    def __init__(self, embedding_dim, hidden_units):
        super(SentenceEncoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.gru = nn.GRU(embedding_dim, hidden_units, bidirectional=True)   # [sentence_length * (2 * hidden_units)]
        self.hidden_state = torch.zeros(2, hidden_units)

    def forward(self, document_embedding):
        outputs, _ = self.gru(document_embedding, self.hidden_state)
        return outputs   # [sentence_length, 2 * hidden_units]

In [ ]:
class SentenceAttention(nn.Module):
    def __init__(self, input_size, hidden_size):
      super(SentenceAttention, self).__init__()
      self.hidden_size = hidden_size
      self.score_weight_matrix = nn.Linear(input_size, hidden_size, bias=True)
      self.softmax = nn.Softmax(dim=1)
      self.context_vector = nn.Parameter(torch.randn(1, hidden_size))

    def forward(self, encoded_sent):
      u_i = torch.tanh(self.score_weight_matrix(encoded_sent))   # [batch_size, T, hidden_size]
      dot_product = torch.matmul(u_i, self.context_vector.transpose(0, 1))  # [batch_size, T, hidden_size]
      a_i = nn.Softmax(dim=0)(dot_product)  # [batch_size, T, hidden_size]
      s_i = (a_i * encoded_sent).sum(dim=1)
      return s_i

In [ ]:
class DocumentClassification(nn.Module):
  def __init__(self, input_size, output_size):
    super(DocumentClassification, self).__init__()
    self.input_size = input_size
    self.output_size = output_size
    self.linear = nn.Linear(input_size, output_size, bias=True)
  def forward(self, encoded_doc):
    p = self.linear(encoded_doc)
    p = nn.Softmax(dim=0)(p)
    return p


In [ ]:
class HAN(nn.Module):
  def __init__(self, sent_length, doc_length, embedding_dim, hidden_units, context_size, output_size):
    super(HAN, self).__init__()
    self.word_encoder = WordEncoder(embedding_dim, hidden_units)
    self.word_att = WordAttention(2 * hidden_units, context_size)
    self.sentence_encoder = SentenceEncoder(sent_length, hidden_units)
    self.sentence_att = SentenceAttention(2 * hidden_units, doc_length)
    self.doc_classify = DocumentClassification(doc_length, output_size)

  def forward(self, doc):
    sentences = [sent.strip() for sent in doc]
    sentences = [WordEmbedding(sentence) for sentence in sentences]
    sentences = pad_sequence(([torch.tensor(sentence) for sentence in sentences]), batch_first=True, padding_value=0)
    document_embedding = []
    for sentence_embedding in sentences:
      encoded_sent = self.word_encoder(sentence_embedding)
      att_sent = self.word_att(encoded_sent)
      document_embedding.append(att_sent)
    document_embedding = torch.stack(document_embedding) # if it needs to be 3D dimensional then add .unsqueeze(1)
    sentence_encoder = self.sentence_encoder(document_embedding)   # 16: max number of word in a sentence
    encoded_doc = self.sentence_att(sentence_encoder)
    predictions = self.doc_classify(encoded_doc)
    return predictions

In [ ]:
han = HAN(sent_length, doc_length, 50, 20, 40, 10)
han(doc)